## Введение в Pandas

In [ ]:
from google.colab import drive

drive.mount('/gdrive' )

In [2]:
import pandas as pd

data = pd.read_csv("/gdrive/My Drive/data/201809-citibike-tripdata.csv")

In [3]:
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1635,2018-09-01 00:00:05.2690,2018-09-01 00:27:20.6340,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,25577,Subscriber,1980,1
1,132,2018-09-01 00:00:11.2810,2018-09-01 00:02:23.4810,314.0,Cadman Plaza West & Montague St,40.693830,-73.990539,3242.0,Schermerhorn St & Court St,40.691029,-73.991834,34377,Subscriber,1969,0
2,3337,2018-09-01 00:00:20.6490,2018-09-01 00:55:58.5470,3142.0,1 Ave & E 62 St,40.761227,-73.960940,3384.0,Smith St & 3 St,40.678724,-73.995991,30496,Subscriber,1975,1
3,436,2018-09-01 00:00:21.7460,2018-09-01 00:07:38.5830,308.0,St James Pl & Oliver St,40.713079,-73.998512,3690.0,Park Pl & Church St,40.713342,-74.009355,28866,Subscriber,1984,2
4,8457,2018-09-01 00:00:27.3150,2018-09-01 02:21:25.3080,345.0,W 13 St & 6 Ave,40.736494,-73.997044,380.0,W 4 St & 7 Ave S,40.734011,-74.002939,20943,Customer,1994,1


In [4]:
data.tail()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
1877879,369,2018-09-30 00:27:25.9840,2018-09-30 00:33:35.7070,3601.0,Sterling St & Bedford Ave,40.662706,-73.956912,3631.0,Crown St & Bedford Ave,40.666563,-73.956741,32976,Subscriber,1985,2
1877880,191,2018-09-30 00:30:30.1850,2018-09-30 00:33:42.1090,3601.0,Sterling St & Bedford Ave,40.662706,-73.956912,3631.0,Crown St & Bedford Ave,40.666563,-73.956741,15595,Subscriber,1985,2
1877881,1442,2018-09-30 08:10:03.1790,2018-09-30 08:34:05.3870,3601.0,Sterling St & Bedford Ave,40.662706,-73.956912,471.0,Grand St & Havemeyer St,40.712868,-73.956981,28646,Subscriber,1981,1
1877882,453,2018-09-30 12:20:13.6830,2018-09-30 12:27:46.9140,3601.0,Sterling St & Bedford Ave,40.662706,-73.956912,3584.0,Eastern Pkwy & Franklin Ave,40.670777,-73.957680,34272,Subscriber,1986,1
1877883,1354,2018-09-30 14:10:01.1620,2018-09-30 14:32:36.1280,3601.0,Sterling St & Bedford Ave,40.662706,-73.956912,3321.0,Clinton St & Union St,40.683116,-73.997853,26028,Subscriber,1981,1


##### *1. Найти общее количество строк и столбцов в датасете - указать первым число строк, вторым - число столбцов*


In [5]:
data.shape

(1877884, 15)

##### *2. Найти среднюю длину поездок в минутах(столбец tripduration) c точностью до 2 знака*

In [6]:
data['tripduration'] = data['tripduration']/60
round(data.describe()['tripduration']['mean'], 2)

16.13

##### *3. Сколько поездок начались и закончились в той же самой станции?*

In [7]:
data[(data['start station id'] == data['end station id'])].shape


(41364, 15)

##### *4. Какой самый используемый байк(bikeid) в городе по количеству поездок?*

In [8]:
data['bikeid'].value_counts()[:1]

33875    559
Name: bikeid, dtype: int64

##### *5. Найдите идентификатор велосипеда (bikeid), у которого в среднем продолжительность поездок выше, чем у всех остальных*



In [9]:
data.groupby('bikeid')['tripduration'].mean().sort_values(ascending=False)[:1]


bikeid
17548    8133.706667
Name: tripduration, dtype: float64

##### *6. Сколько строк, в которых отсутствуют данные о start station id?*



In [10]:
data[data['start station id'].isnull()].shape


(716, 15)

##### *7. Какова средняя продолжительность поездки в минутах в зависимости от типа подписки c точностью до 2 знака?*



In [11]:
round(data.groupby('usertype')['tripduration'].mean(),2)


usertype
Customer      33.42
Subscriber    13.33
Name: tripduration, dtype: float64

##### *8. Найдите среднюю длину поездок в километрах с точностью до 2 знака, предварительно выкинув замкнутые траектории(те у которых совпадают start station id = end station id).*



In [12]:
from geopy.distance import Point, vincenty


data['point1'] = data.apply(lambda x: Point(latitude=x['start station latitude'], longitude=x['start station longitude']), axis=1)
data['point2'] = data.apply(lambda y: Point(latitude=y['end station latitude'], longitude=y['end station longitude']), axis=1)

data['distance_km'] = data.apply(lambda z: vincenty(z['point1'], z['point2']).kilometers if z['point2'] is not z['point1'] else float('nan'), axis=1)
data = data.drop('point2', axis=1)

round(data['distance_km'].mean(),2)

  

1.81

##### *9. Выберите станцию (start station id) с максимальным количеством отправлений с 18 до 20 вечера включительно*



In [13]:
import datetime
import time

data['start_hour'] = data['starttime'].apply(lambda x: datetime.datetime.fromtimestamp(
    time.mktime(datetime.datetime.strptime(x.strip(), "%Y-%m-%d %H:%M:%S.%f").timetuple())).hour)

In [14]:
data[(data.start_hour.isin([18, 19, 20]))]['start station id'].value_counts().head(1)

519.0    3181
Name: start station id, dtype: int64

##### *10. Выберите идентификаторы станций(end station id), в которые приезжают с 6 до 10 утра включительно*



In [15]:
data['end_hour'] = data['stoptime'].apply(lambda x: datetime.datetime.fromtimestamp(
            time.mktime(datetime.datetime.strptime(x.strip(), "%Y-%m-%d %H:%M:%S.%f").timetuple())).hour)

In [16]:
values = [3140, 3106, 3116, 369]

In [17]:
data[(data.end_hour.isin([6, 7, 8, 9, 10])) & 
     (data['end station id'].isin(values))]['end station id'].unique()

array([3106., 3116., 3140.,  369.])